# Pour P., lister les limites de qualités dans la data
***
_Les parties indispensables à run sont indiquées avec la mention [TO RUN], afin de reproduire les analyses se situant sous ces sections._

**Contexte** :<br>
Il y aurait 467 paramètres sans limite de qualité dans nos données qui ont été identifiés<br>
On veut fournir à Pauline cette liste


**Choses à faire** :
- Sur les données 2024, recroiser avec les catégories de Pauline, et ajouter une colonne à son fichier avec la (ou les) limites trouvées (distinctes).
- Si pas de limite trouvée, laisser vide.
- Notebook qui exporte un fichier Excel


# Résumé de l'étude
***

Je trouve **540 paramètres sans limite de qualité** ( limitequal IS NULL  ) dans la table edc_resultats complète (sans filtre sur l'année)<br>
et 403 paramètres sans limite de qualité en 2024 uniquement.


J'importe la liste dans un fichier excel (**D4G_limite_de_qualite.xlsx**) pour Pauline

# Etude
***
### Packages + read data

In [1]:
# [TO RUN]  Packages
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [2]:
# [TO RUN]
import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [3]:
!uv pip install openpyxl # pour pd.to_excel

Using Python 3.12.9 environment at: C:\Users\Vinca\Documents\D4G_2025\13_pollution_eau\.venv
Audited 1 package in 44ms


###  Exploration

In [4]:
# [OPTIONAL] Tables dispo dans la bdd
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ mapping_categories      │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
└─────────────────────────┘



In [5]:
# [OPTIONAL] Preview edc_resultats
preview_resultats = con.sql("SELECT * FROM edc_resultats LIMIT 2").df()
preview_resultats

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.0,107-06-2,00100125759,2020,2025-02-14,20230811-150005
1,001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.0,2163-68-0,00100125759,2020,2025-02-14,20230811-150005


# Identification des paramètre sans limite de qualité

In [6]:
con.sql("SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL").show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         540 │
└─────────────────────────────┘



In [7]:
con.sql("SELECT COUNT(DISTINCT cdparametre ) FROM   edc_resultats WHERE limitequal IS NULL AND de_partition='2024'").show()

┌─────────────────────────────┐
│ count(DISTINCT cdparametre) │
│            int64            │
├─────────────────────────────┤
│                         403 │
└─────────────────────────────┘



In [8]:
query_no_param_limitequal = """ 
SELECT 
  cdparametre,
  STRING_AGG(DISTINCT libmajparametre) AS list_libmajparametre  ,
  STRING_AGG(DISTINCT cdparametresiseeaux) AS list_cdparametresiseeaux,
  STRING_AGG(DISTINCT casparam) AS list_casparam,
FROM  
    edc_resultats 
WHERE
    limitequal IS NULL 
    AND cdparametre IS NOT NULL
GROUP BY 
    cdparametre
"""

df_no_param_limitequal = con.sql(query_no_param_limitequal).df()
df_no_param_limitequal

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cdparametre,list_libmajparametre,list_cdparametresiseeaux,list_casparam
0,5901,ODEUR (QUALITATIF),ODQ,None
1,1498,"DIBROMOÉTHANE-1,2",12BRE,106-93-4
2,1513,DIBROMOMÉTHANE,12DBM,74-95-3
3,2611,CHLOROPRÈNE,CLPREN,126-99-8
4,1541,STYRÈNE,STYR,100-42-5
...,...,...,...,...
535,2787,TRIPHENYL PHOSPHATE,TRIPHOS,115-86-6
536,1082,BENZANTHRACÈNE,BENZAN,56-55-3
537,1342,SILICATES (EN MG/L DE SIO2),SIL,15593-90-5
538,1056,ENTÉROVIRUS,EVIR,None


In [9]:
# To excel
df_no_param_limitequal.to_excel("D4G_limite_de_qualite.xlsx",
              sheet_name='parametre_sans_limite')  

# Identification des limites de qualité dans les données

In [10]:
# [TO RUN]

query_check_limitequal = """ 
WITH CAT AS (
    SELECT
      libmajparametre,
      categorie,
    FROM
      mapping_categories
)

SELECT 
  CAT.categorie AS cat_polluant,
  edc_resultats.cdparametre AS cdparametre,
  edc_resultats.limitequal,
  CAST(regexp_extract(REPLACE("limitequal", ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
  regexp_extract("limitequal", '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite
FROM  
    edc_resultats 
LEFT JOIN
    CAT
ON 
  edc_resultats.libmajparametre =   CAT.libmajparametre
GROUP BY
    1,2,3
HAVING 
    cat_polluant IS NOT NULL
    AND limitequal IS NOT NULL   
    AND cat_polluant != 'non classé'
"""

con.sql(query_check_limitequal).show()

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_7512\782829828.py:3: SyntaxWarning: invalid escape sequence '\d'
  query_check_limitequal = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────────────────┬─────────────┬─────────────┬──────────────────┬─────────┐
│       cat_polluant        │ cdparametre │ limitequal  │ limitequal_float │  unite  │
│          varchar          │    int64    │   varchar   │      float       │ varchar │
├───────────────────────────┼─────────────┼─────────────┼──────────────────┼─────────┤
│ pesticides                │        1808 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        5507 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        2974 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        1263 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        2664 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        2085 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ métabolite de pesticide   │        1954 │ <=0,1 µg/L  │              0.1 │ µg/L    │
│ pesticides                │        7511 │

In [13]:
# [TO RUN]

query_check_limitequal_cat = """ 
WITH CAT AS (
    SELECT
      libmajparametre,
      categorie,
    FROM
      mapping_categories
)

SELECT DISTINCT
  CAT.categorie AS cat_polluant,
  edc_resultats.limitequal,
  CAST(regexp_extract(REPLACE("limitequal", ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
  regexp_extract("limitequal", '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite
FROM  
    edc_resultats 
LEFT JOIN
    CAT
ON 
  edc_resultats.libmajparametre =   CAT.libmajparametre
WHERE 
    cat_polluant IS NOT NULL
    AND limitequal IS NOT NULL   
    AND cat_polluant != 'non classé'
ORDER BY
    cat_polluant    
"""

df_check_limitequal_cat = con.sql(query_check_limitequal_cat).df()
df_check_limitequal_cat

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_7512\2099144781.py:3: SyntaxWarning: invalid escape sequence '\d'
  query_check_limitequal_cat = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cat_polluant,limitequal,limitequal_float,unite
0,cvm,<=0.5 µg/L,0.50,µg/L
1,hydrocarbure,<=3 µg/L,3.00,µg/L
2,hydrocarbure,<=10 µg/L,10.00,µg/L
3,hydrocarbure,<=1 µg/L,1.00,µg/L
4,hydrocarbure,<=0.01 µg/L,0.01,µg/L
5,hydrocarbure,<=0.1 µg/L,0.10,µg/L
6,hydrocarbure,<=5 NG/(KG POIDS CORPOREL/J),5.00,
7,hydrocarbure,"<=0,1 µg/L",0.10,µg/L
8,hydrocarbures,<=10 µg/L,10.00,µg/L
9,microbio,<=1 µg/L,1.00,µg/L


<b>Les textes sur les limites de qualité</b>

Directive 98/83/CE du Conseil du 3 novembre 1998 relative à la qualité des eaux destinées à la consommation humaine<br>
https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000000521549

Directive 2020/2184 du Parlement européen et du Conseil du 16 décembre 2020 relative à la qualité des eaux destinées à la consommation humaine: <br>
https://sante.gouv.fr/sante-et-environnement/eaux/article/une-nouvelle-directive-eau-potable

L'annexe II de l' Arrêté du 11 janvier 2007 relatif aux limites et références de qualité des eaux brutes et des eaux destinées à la consommation humaine mentionnées aux articles R. 1321-2, R. 1321-3, R. 1321-7 et R. 1321-38 du code de la santé publique <br>
https://www.legifrance.gouv.fr/loda/article_lc/LEGIARTI000035438978<br>
--> Table disponible

Arrêté du 30/12/22 modifiant l'arrêté du 11 janvier 2007 relatif aux limites et références de qualité des eaux brutes et des eaux destinées à la consommation humaine mentionnées aux articles R. 1321-2, R. 1321-3, R. 1321-7 et R. 1321-38 du code de la santé publique<br>
https://aida.ineris.fr/reglementation/arrete-301222-modifiant-larrete-11-janvier-2007-relatif-limites-references-qualite<br>
--> Table disponible


Pour les métaux lourds : on a des limites différentes selon le métal

| Métal | Limite de qualité | Unité | 
| --- | --- | --- | 
|Arsenic|	10|	µg/L|
|Cadmium|	5	|µg/L|
|Chrome|	50|	µg/L|
|Mercure|	1	|µg/L|
|Nickel	|20	|µg/L|
|Plomb|	10	|µg/L|
|Sélénium|	10	|µg/L|

De même pour les mineraux et les Sous-produits de désinfection

<b>Proposition de table</b><br>

| catégorie | Limite de qualité en µg/L  | 
| --- | --- | 
|pesticides|	2	|
|cvm	|0,5	|
|métabolite de pesticide	|5	|
|pcb	|0,5|
|perchlorate	|4	|
|phénol|	0,5|
|nitrite	|100|
|pfas	|0,1|
|phtalate	|0,1|

In [12]:
# [TO RUN] Création d'une table avec les limites de qualité par catégorie

limite_quality = {
    "categorie": [
        "pesticides",
        "cvm",
        "métabolite de pesticide",
        "pcb",
        "perchlorate",
        "phénol",
        "nitrite",
        "pfas",
        "phtalate",
    ],
    "Limite_qual": [2, 0.5, 5, 0.5, 4, 0.5, 100, 0.1, 0.1],
}

limite_quality_df = pd.DataFrame(limite_quality)
limite_quality_df

,categorie,Limite_qual
0,pesticides,2.0
1,cvm,0.5
2,métabolite de pesticide,5.0
3,pcb,0.5
4,perchlorate,4.0
5,phénol,0.5
6,nitrite,100.0
7,pfas,0.1
8,phtalate,0.1
